# Simulation analysis: 

This notebook illustrates how we can combine several libraries (pyvista, numpy, matplotlib, etc.) to carry out some basic N-dimensional data analysis.

## Study case: Turbulence-in-a-box in 2D

The study case is a set of data produced by a numerical (magnetohydrodynamical) simulation of 2D turbulence in a periodic domain. The gas in this simulation is isothermal, so it is described by this equation of state:

$$p = \rho\,c_{\rm iso}^2$$

where $\rho$ is the gas density and $c_{\rm iso}$ is its isothermal sound speed.

In this simulation, the gas is initially still and has a uniform density distribution. Then, turbulence is generated by appying a time-dependent, spatially-varying, stochastic force field. Such force field continuously injects mechanical energy into the box and maintains supersonic turbulence during the whole simulation.

The output files from the simulation are saved as VTK files:

- At low resolution (recommended):
https://drive.google.com/file/d/1q86eBofXP5d8ljg0NJp944Y7WXtMxXjh/view?usp=drive_link

- At high resolution:
https://drive.google.com/file/d/1q5Y5N_kTFn0Y8bnfcIbgBdZ809UqgHTD/view?usp=drive_link


Each VTK file contains several data arrays, which we want to analyse and visualise. In addition, these files are accompanied by additional descriptor files that contain information on"

- the physical (CGS) normalisation units of the problem, see **units.out**

- the simulation times, see second column of **vtk.out**.

- additional grid information, see **grid.out**

The structure of these files makes this problem ideal to practice data/image analysis.


### Physical units:

The VTK data files are written in code units by default, so we need to use the information from **units.out** to convert the data from code to CGS units. The fundamental normalisation parameters are:

- length, $L_0$

- velocity, $v_0$

- density, $\rho_0$

To convert from code units to physical units, we can multiply the variables by the normalisation parameters in CGS units, e.g. to get the density in CGS units:


$L_{cgs} = L\times L_0$, where $L$ is the length of the box in code units and $L_0$ comes from the **units.out** file.

$v_{cgs} = v\times v_0$, where $L$ is the velocity in code units and $v_0$ comes from the **units.out** file.


$\rho_{cgs} = \rho\times\rho_0$, where $\rho$ comes from the simulation and $\rho_0$ comes from the **units.out** file.

### Derived physical units:

Using the fundamental normalisation parameters, we can also find the derived normalisation parameters for:

- pressure, $p_0=\rho_0v_0^2$

- magnetic flux density, $B_0 = v_0\,\sqrt{4\,\pi\,\rho_0}$

Thus, to convert pressures and magnetic flux densities to CGS units, we proceed as follows:

$p_{cgs} = p\times p_0$,

$B_{cgs} = B\times B_0$.

### Time units:

In addition, the times can be retrieved from the second column of the **vtk.out** file in code units. To normalise them to CGS units, we can first calculate the normalisation factor for time as:

$t_0 = \frac{L_0}{v_0}$


## Tasks:

#### 1. Read the units.out file and assign the fundamental CGS normalisation parameters to python objects.


#### 2. Calculate the derived CGS normalisation parameters for pressure, magnetic flux density, and time.


#### 3. Read the second column of the vtk.out file and extract the simulation times in code units.  Then, normalise them to CGS units.


#### 4. Choose one of the VTK files (e.g. #10), open it, convert all the data arrays to CGS units, and save them into python objects.


#### 5. Calculate the thermal pressure of the gas in CGS units, using the isothermal EOS.


#### 6. Make the following maps in CGS units:

- Density + velocity vector field

- Thermal pressure

- Magnetic flux density magnitude + its vector field

#### 7. Make a histogram of the density field and fit an appropriate function using regression methods.


#### 8. Make a 2D histogram of pressure versus density, and check if it folows the expected EOS.


#### 8. Create a function that calculates the following quantities and prints them into a pandas data frame with time in the first column and headers:

 - density average 
 
 - density standard deviation
 
 - thermal pressure average
 
 - thermal pressure standard deviation

#### 1. Read the units.out file and assign the fundamental CGS normalisation parameters to python objects.

In [1]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
import pyvista as pv
import pandas as pd

In [4]:
# Read units.out file

df_units = pd.read_csv("/Users/wladimir/Dropbox/Sample_Data/TURB_DRIVE_SUP/units.out")

print(df_units)

  variable  normalisation   units
0    rho_0   1.660000e-24  g/cm^3
1      v_0   1.000000e+06    cm/s
2      L_0   3.086000e+18      cm
3    c_iso   1.000000e+06    cm/s


In [8]:
# Place these values into python objects:

rho_0 = np.array(df_units.loc[df_units["variable"] == "rho_0"]["normalisation"])
v_0 = np.array(df_units.loc[df_units["variable"] == "v_0"]["normalisation"])
L_0 = np.array(df_units.loc[df_units["variable"] == "L_0"]["normalisation"])
c_iso = np.array(df_units.loc[df_units["variable"] == "c_iso"]["normalisation"])

print(rho_0, v_0, L_0, c_iso)

[1.66e-24] [1000000.] [3.086e+18] [1000000.]


#### 2. Calculate the derived CGS normalisation parameters for pressure, magnetic flux density, and time.

In [10]:
# Let's get t_0 in CGS units
t_0 = L_0/v_0

# Let's get the pressure in CGS units
p_0 = rho_0*v_0**2

# Let's get B_0
B_0 = v_0*np.sqrt(4*np.pi*rho_0)

print(t_0, p_0, B_0)

[3.086e+12] [1.66e-12] [4.56729408e-06]


#### 3. Read the second column of the vtk.out file and extract the simulation times in code units.  Then, normalise them to CGS units.

In [13]:
# Use pandas to retrieve times

df_times = pd.read_csv("/Users/wladimir/Dropbox/Sample_Data/TURB_DRIVE_SUP/vtk.out", sep = "\s+", header = None)

print(df_times)

      0         1         2    3            4       5    6    7    8    9   \
0      0  0.000000  0.000100    0  single_file  little  rho  vx1  vx2  Bx1   
1      1  0.018194  0.001919   31  single_file  little  rho  vx1  vx2  Bx1   
2      2  0.036404  0.003740   38  single_file  little  rho  vx1  vx2  Bx1   
3      3  0.059240  0.006024   43  single_file  little  rho  vx1  vx2  Bx1   
4      4  0.079180  0.008018   46  single_file  little  rho  vx1  vx2  Bx1   
..   ...       ...       ...  ...          ...     ...  ...  ...  ...  ...   
96    96  1.918657  0.001664  809  single_file  little  rho  vx1  vx2  Bx1   
97    97  1.938610  0.001678  821  single_file  little  rho  vx1  vx2  Bx1   
98    98  1.958715  0.001762  833  single_file  little  rho  vx1  vx2  Bx1   
99    99  1.979934  0.001973  844  single_file  little  rho  vx1  vx2  Bx1   
100  100  2.000000  0.000444  855  single_file  little  rho  vx1  vx2  Bx1   

      10  
0    Bx2  
1    Bx2  
2    Bx2  
3    Bx2  
4    Bx2

In [18]:
# Get the second column
time_code = np.array(df_times.iloc[:,1])
#print(time_code)

# Convert this to CGS units
t_cgs = time_code*t_0

print(t_cgs)

[0.00000000e+00 5.61477332e+10 1.12343793e+11 1.82814856e+11
 2.44348030e+11 2.96175795e+11 3.46957437e+11 4.15590694e+11
 4.78889800e+11 5.53238021e+11 6.04333232e+11 6.66554398e+11
 7.37688858e+11 7.91171398e+11 8.63854105e+11 9.20062817e+11
 9.83681325e+11 1.04785284e+12 1.10116936e+12 1.16971713e+12
 1.22553300e+12 1.29236063e+12 1.35046384e+12 1.41711095e+12
 1.47374707e+12 1.53822349e+12 1.60097576e+12 1.65960142e+12
 1.72521071e+12 1.78316147e+12 1.84764529e+12 1.90767385e+12
 1.96969350e+12 2.03566632e+12 2.09623401e+12 2.15974049e+12
 2.21639236e+12 2.27974917e+12 2.34371022e+12 2.40255808e+12
 2.46390838e+12 2.52541360e+12 2.59189128e+12 2.65099034e+12
 2.71055971e+12 2.77197234e+12 2.83425369e+12 2.89533581e+12
 2.95788934e+12 3.01652920e+12 3.07868279e+12 3.14494877e+12
 3.20286065e+12 3.26745989e+12 3.32904719e+12 3.38970252e+12
 3.45401784e+12 3.51561440e+12 3.57676041e+12 3.63871803e+12
 3.69690147e+12 3.76344181e+12 3.82570186e+12 3.88578010e+12
 3.94556210e+12 4.006488